# Лабораторная работа 1

### Разделение массива на количественный и качественный

In [2]:
# импорт библиотек
import numpy as np
import pandas as pd
import scipy.stats as stats

In [3]:
# выгрузка данных
df_0 = pd.read_csv('V11.csv', sep=';', decimal=',')
df_0.head()

,A6,A10,A13,A16,A18,A20,A21,A22,A23,A37,A40
0,key,coated,line,821,tabloid,1911,25.0,38,0.200,40,noband
1,key,coated,line,816,spiegel,1910,45.0,54,0.167,35,noband
2,key,coated,line,827,tabloid,1911,50.0,46,0.200,40,noband
3,key,uncoated,line,802,catalog,1910,47.5,52,0.267,33,band
4,key,uncoated,line,815,catalog,1911,50.0,60,0.300,40,band


In [4]:
# разделим на 2 датафрейма - качественные и количественные данные
df_1 = df_0.iloc[:, 0:6]
df_1['A40'] = df_0['A40']
#df_1.head()

df_2 = df_0.iloc[:, 6:10]
df_2['A40'] = df_0['A40']
df_2.head()

,A21,A22,A23,A37,A40
0,25.0,38,0.200,40,noband
1,45.0,54,0.167,35,noband
2,50.0,46,0.200,40,noband
3,47.5,52,0.267,33,band
4,50.0,60,0.300,40,band


In [6]:
# Произведем калибровку качественных данных с учетом априорного шанса с поправкой Лапласа
# С помощью формулы мы получили числа в таблице Excel, вставим их в дф, изменив старые значения

# A6
replace_dict = {'key': 0.001311808, 'type': 1}
df_1['A6'] = df_1['A6'].replace(replace_dict)

# A10
replace_dict = {'coated': 0.00200664, 'uncoated': 0.002898667, 'super': 1}
df_1['A10'] = df_1['A10'].replace(replace_dict)

# A13
replace_dict = {'line': 0.001308837, 'xylol': 0.176470588, 'lactol': 1, 'naptha': 0.076923077, 'other': 1}
df_1['A13'] = df_1['A13'].replace(replace_dict)

# A16
replace_dict = {821: 0.021956088, 802: 0.007407407, 813: 0.005544006, 
                824: 0.003076923, 815: 0.016248154, 816: 0.00752909, 
                827: 0.009784736, 828: 0.011124845
}
df_1['A16'] = df_1['A16'].replace(replace_dict)

# A18
replace_dict = {'tabloid': 0.00255052, 'spiegel': 0.009983361, 'catalog': 0.003649231}
df_1['A18'] = df_1['A18'].replace(replace_dict)

# A20
replace_dict = {1911: 0.004417834, 1910: 0.001846365, 'other': 1}
df_1['A20'] = df_1['A20'].replace(replace_dict)

df_1.head()

,A6,A10,A13,A16,A18,A20,A40
0,0.001312,0.002007,0.001309,0.021956,0.002551,0.004418,noband
1,0.001312,0.002007,0.001309,0.007529,0.009983,0.001846,noband
2,0.001312,0.002007,0.001309,0.009785,0.002551,0.004418,noband
3,0.001312,0.002899,0.001309,0.007407,0.003649,0.001846,band
4,0.001312,0.002899,0.001309,0.016248,0.003649,0.004418,band


In [7]:
# Проведем дискретизацию с равной частотой количественных данных
quantiles = [0, 0.2, 0.4, 0.6, 0.8, 1]

for column in df_2:
    if column != 'A40':

        # Получение уникальных квантилей
        q = df_2[column].quantile(quantiles)
        unique_quantiles = q.unique()
        print(unique_quantiles)

        # этот массив потом вставить можно в дф
        arr = []

        # сравнение каждого значения дф с границами инт-лов
        for value in df_2[column]:
            for i in range(len(unique_quantiles)):
                if i == 0 and unique_quantiles[i] <= value <= unique_quantiles[i+1]:
                    arr.append(i+1)
                    break
                if unique_quantiles[i] < value <= unique_quantiles[i+1]:
                    arr.append(i+1)

        # создание нового столбца
        df_2[f'{column}_dis'] = arr
        

# Вывод результата
print(df_2)



[25.  37.5 40.  47.  50.  72.5]
[35. 43. 48. 54. 58. 72.]
[0.133 0.2   0.267 0.3   0.333 0.533]
[30. 35. 40. 45.]
      A21  A22    A23  A37     A40  A21_dis  A22_dis  A23_dis  A37_dis
0    25.0   38  0.200   40  noband        1        1        1        2
1    45.0   54  0.167   35  noband        3        3        1        1
2    50.0   46  0.200   40  noband        4        2        1        2
3    47.5   52  0.267   33    band        4        3        2        1
4    50.0   60  0.300   40    band        4        5        3        2
..    ...  ...    ...  ...     ...      ...      ...      ...      ...
495  30.0   54  0.500   35  noband        1        3        5        1
496  52.5   62  0.433   35    band        5        5        5        1
497  52.0   40  0.300   40  noband        5        1        3        2
498  60.0   56  0.300   40    band        5        4        3        2
499  45.0   53  0.300   35  noband        3        3        3        1

[500 rows x 9 columns]


In [8]:
# преобразуем A40 в столбец с 0 и 1 в обоих дф
df_1['A40'] = df_1['A40'].apply(lambda x: 1 if x == 'band' else 0)
df_2['A40'] = df_2['A40'].apply(lambda x: 1 if x == 'band' else 0)
print(df_1)
print(df_2)

           A6       A10       A13       A16       A18       A20  A40
0    0.001312  0.002007  0.001309  0.021956  0.002551  0.004418    0
1    0.001312  0.002007  0.001309  0.007529  0.009983  0.001846    0
2    0.001312  0.002007  0.001309  0.009785  0.002551  0.004418    0
3    0.001312  0.002899  0.001309  0.007407  0.003649  0.001846    1
4    0.001312  0.002899  0.001309  0.016248  0.003649  0.004418    1
..        ...       ...       ...       ...       ...       ...  ...
495  0.001312  0.002007  0.001309  0.003077  0.002551  0.001846    0
496  0.001312  0.002899  0.001309  0.021956  0.002551  0.001846    1
497  0.001312  0.002899  0.001309  0.007529  0.003649  0.001846    0
498  0.001312  0.002899  0.176471  0.007529  0.003649  0.004418    1
499  0.001312  0.002007  0.001309  0.007529  0.002551  0.001846    0

[500 rows x 7 columns]
      A21  A22    A23  A37  A40  A21_dis  A22_dis  A23_dis  A37_dis
0    25.0   38  0.200   40    0        1        1        1        2
1    45.0   

### Вычисление взаимной информации, коэфициента неопределенности и симметричного коэфициента

In [9]:
# импорт модулей
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
from sklearn.feature_selection import mutual_info_classif

In [24]:
# сливаем количественный и качественный дф в один (пока не уверена, что это надо делать, поэтому не делала)


# рассчитываем показатель взаимной информации, коэф-т неопределенности, симметричный коэф-т
def calculate_associations(df):

    # дфы для результатов
    df_mi = pd.DataFrame(index=df.columns, columns=df.columns) # взаимн. инф.
    df_uc = pd.DataFrame(index=df.columns, columns=df.columns) # коэф. неопр-ти
    df_su = pd.DataFrame(index=df.columns, columns=df.columns) # симметричный коэф.

    # перебираем все пары столбцов
    for col1 in df.columns:
        for col2 in df.columns:
            # одинаковые столбцы пропускаем
            if col1 == col2:
                continue

            # рассчитываем показатели взаимосвязи с помощью sklearn и согласно формулам из лекции
            mutual_info = mutual_info_score(df[col1], df[col2])
            uncert_coef = mutual_info_score(df[col1], df[col2]) / entropy(df[col2])
            symm_uncert = 2 * mutual_info / (entropy(df[col1]) + entropy(df[col2]))

            # записываем результаты в дф
            df_mi.loc[col1, col2] = mutual_info
            df_uc.loc[col1, col2] = uncert_coef
            df_su.loc[col1, col2] = symm_uncert

    return df_mi, df_uc, df_su

# ассчитываем показатели взаимосвязи
df_mi, df_uc, df_su = calculate_associations(df_1)

c:\Users\Екатерина\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\Users\Екатерина\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\Users\Екатерина\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\Users\Екатерина\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering m

In [30]:
print(df_mi)
print("\n\n", df_uc)
print("\n\n", df_su)

      A6       A10       A13       A16       A18       A20       A40
A6   NaN       0.0       0.0       0.0       0.0       0.0       0.0
A10  0.0       NaN  0.019542  0.144514  0.128728  0.003129  0.018751
A13  0.0  0.019542       NaN  0.039014  0.020615  0.007919   0.01204
A16  0.0  0.144514  0.039014       NaN  0.460486  0.032215  0.063722
A18  0.0  0.128728  0.020615  0.460486       NaN  0.004746  0.001442
A20  0.0  0.003129  0.007919  0.032215  0.004746       NaN  0.001551
A40  0.0  0.018751   0.01204  0.063722  0.001442  0.001551       NaN


       A6       A10       A13       A16       A18       A20       A40
A6   NaN       0.0       0.0       0.0       0.0       0.0       0.0
A10  0.0       NaN  0.004977  0.023807  0.021159  0.000511  0.003546
A13  0.0  0.003152       NaN  0.006427  0.003389  0.001294  0.002277
A16  0.0   0.02331  0.009937       NaN  0.075691  0.005263   0.01205
A18  0.0  0.020764  0.005251   0.07586       NaN  0.000775  0.000273
A20  0.0  0.000505  0.002017  0